In [1]:
import pandas as pd

In [16]:
model = 'CRYPTOBERT'

In [17]:
crypto = 'crypto_bitcoin'
# crypto = 'crypto_ethereum'

In [18]:
list_subs = ['bitcoin','bitcoinbeginners','bitcoinmarkets','bitcoinmining','btc']
# list_subs = ['ethereum','ethermining','ethfinance','eth','ethtrader']

In [20]:
dfs = []
for subreddit in list_subs:
    dfs.append(pd.read_csv(f"Data/Sentiment/{crypto}/{model}/{subreddit}_sentiment_submission_19_22.csv"))

In [21]:
final_df = pd.concat(dfs)

In [22]:
final_df['subreddit'].unique()

array(['Bitcoin', 'BitcoinBeginners', 'BitcoinMarkets', 'BitcoinMining',
       'btc'], dtype=object)

In [23]:
final_df.columns

Index(['submission', 'post_text', 'subreddit', 'score', 'num_comments',
       'posted_on', '# Of Words', 'label', 'score.1'],
      dtype='object')

In [24]:
final_df['posted_on'] = pd.to_datetime(final_df['posted_on'])

# Set the 'posted_on' as the index
final_df.set_index('posted_on', inplace=True)

In [ ]:
if model == 'VADER':
    final_df['VADER_Post_Sentiment'] = pd.Categorical(final_df['VADER_Post_Sentiment'], categories=['Positive', 'Negative', 'Neutral'])

    # Create a DataFrame to store the aggregated statistics
    aggregated_df = pd.DataFrame()

    # Count the occurrences of each sentiment category per day
    aggregated_df['positive_count'] = final_df['VADER_Post_Sentiment'].eq('Positive').groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['negative_count'] = final_df['VADER_Post_Sentiment'].eq('Negative').groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['neutral_count'] = final_df['VADER_Post_Sentiment'].eq('Neutral').groupby(pd.Grouper(freq='D')).sum()
    
    # Aggregating other features
    aggregated_df['total_posts'] = final_df['submission'].groupby(pd.Grouper(freq='D')).size()
    aggregated_df['total_words'] = final_df['# Of Words'].groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['total_stopwords'] = final_df['# Of StopWords'].groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['total_comments'] = final_df['num_comments'].groupby(pd.Grouper(freq='D')).sum()

    # Mean of 'Average Word Length' per day
    aggregated_df['average_word_length'] = final_df['Average Word Length'].groupby(pd.Grouper(freq='D')).mean()

    # Mean of 'VADER_Score' per day
    aggregated_df['mean_vader_score'] = final_df['VADER_Score'].groupby(pd.Grouper(freq='D')).mean()

    # Fill NaN values with 0
    aggregated_df.fillna(0, inplace=True)

    # # Print the aggregated DataFrame
    display(aggregated_df.head(4))

In [25]:
if model == 'CRYPTOBERT':
    final_df['label'] = pd.Categorical(final_df['label'], categories=['Positive', 'Negative', 'Neutral'])

    # Create a DataFrame to store the aggregated statistics
    aggregated_df = pd.DataFrame()

    # Count the occurrences of each sentiment category per day
    aggregated_df['positive_count'] = final_df['label'].eq('Positive').groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['negative_count'] = final_df['label'].eq('Negative').groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['neutral_count'] = final_df['label'].eq('Neutral').groupby(pd.Grouper(freq='D')).sum()

    # Aggregating other features
    aggregated_df['total_posts'] = final_df['submission'].groupby(pd.Grouper(freq='D')).size()
    aggregated_df['total_words'] = final_df['# Of Words'].groupby(pd.Grouper(freq='D')).sum()
    aggregated_df['total_comments'] = final_df['num_comments'].groupby(pd.Grouper(freq='D')).sum()

    # Fill NaN values with 0
    aggregated_df.fillna(0, inplace=True)

    # Print the aggregated DataFrame
    display(aggregated_df.head(4))

,positive_count,negative_count,neutral_count,total_posts,total_words,total_comments
posted_on,,,,,,
2019-01-01,23,3,37,63,8369,989
2019-01-02,35,7,48,90,12730,1150
2019-01-03,44,11,58,113,15544,1784
2019-01-04,31,3,61,95,9832,1388


In [26]:
# Resetting index to make 'posted_on' a column
aggregated_df.reset_index(inplace=True)

In [27]:
aggregated_df.isna().sum()

posted_on         0
positive_count    0
negative_count    0
neutral_count     0
total_posts       0
total_words       0
total_comments    0
dtype: int64

### Saving the Count Time Series

In [28]:
aggregated_df.to_csv(f"Data/Time_Series/{crypto}/{model}_sts_19_22.csv",index=False)